In [ ]:
# 查看当前挂载的数据集目录
!ls /home/kesci/input/

In [ ]:
# 查看个人持久化工作区文件
!ls /home/kesci/work/

In [ ]:
# 查看当前kernerl下的package
!pip list --format=columns

In [1]:
# -*- coding:utf-8 -*-

import os
# os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"  
# os.environ["CUDA_VISIBLE_DEVICES"] = "0,1"

import tensorflow as tf

from tensorflow.contrib import rnn

import tensorflow.contrib.layers as layers
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedShuffleSplit

import tensorflow as tf

import pandas as pd
import numpy as np
from gensim.models import Word2Vec

"""wd_5_bigru_cnn

在论文 Recurrent Convolutional Neural Networks for Text Classification 中。

使用 BiRNN 处理，将每个时刻的隐藏状态和原输入拼起来，在进行 max_pooling 操作。

这里有些不同，首先也是使用 bigru 得到每个时刻的，将每个时刻的隐藏状态和原输入拼起来；

然后使用输入到 TextCNN 网络中。

"""





class Settings(object):

    def __init__(self):


        self.model_name = 'wd_1_2_cnn_max'
        self.query_len = 20
        self.title_len = 20
        self.filter_sizes = [2, 3, 4, 5]
        self.n_filter = 256
        self.fc_hidden_size = 1024
        self.n_class = 1
        self.train_batch_size=512
        self.vaild_batch_size=512
        self.lr=0.0025
        self.decay_step=20000
        self.decay_rate=0.8





class TEXTCNN(object):

    def __init__(self,  settings):


        self.model_name = settings.model_name
        self.query_len = settings.query_len
        self.title_len = settings.title_len
        self.filter_sizes = settings.filter_sizes
        self.n_filter = settings.n_filter
        self.n_filter_total = self.n_filter * len(self.filter_sizes)
        self.n_class = settings.n_class
        self.fc_hidden_size = settings.fc_hidden_size
        self._global_step = tf.Variable(0, trainable=False, name='Global_Step')
        self.update_emas = list()
        # placeholders
        self.training = tf.placeholder(tf.bool)
        self._keep_prob = tf.placeholder(tf.float32, [])
        self._batch_size = tf.placeholder(tf.int32, [])
        self.embedding_size=100
        self.w=4
        self.di = 16
        self.num_layers = 2
        self.model_type="ABCNN3"
        self.eclipse = 1e-10
        

        with tf.name_scope('Inputs'):
            self.embedded_x_query=tf.placeholder(tf.float32,[None,self.query_len,self.embedding_size])#h r
            self.embedded_x_title=tf.placeholder(tf.float32,[None,self.title_len,self.embedding_size])# r
            self._y_inputs = tf.placeholder(tf.float32, [None, self.n_class], name='y_input')

        output_title = self.cnn_inference(self.embedded_x_query, self.query_len,'query')
        output_title = tf.expand_dims(output_title, 0)

        output_content = self.cnn_inference(self.embedded_x_title, self.title_len,'title')
        output_content = tf.expand_dims(output_content, 0)


        with tf.variable_scope('fc-bn-layer'):
            output = tf.concat([output_title, output_content], axis=0)
            output = tf.reduce_max(output, axis=0)
            W_fc = self.weight_variable([self.n_filter_total, self.fc_hidden_size], name='Weight_fc')
            h_fc = tf.matmul(output, W_fc, name='h_fc')
            fc_bn=tf.layers.batch_normalization(h_fc,training=self.training)
            # self.update_emas.append(update_ema_fc)
            self.fc_bn_relu = tf.nn.relu(fc_bn, name="relu")
            fc_bn_drop = tf.nn.dropout(self.fc_bn_relu, self.keep_prob)

        with tf.variable_scope('out_layer'):
            W_out = self.weight_variable([self.fc_hidden_size, self.n_class], name='Weight_out')
            b_out = self.bias_variable([self.n_class], name='bias_out')
            self.sim = self.lcnn_foward()
            self._y_pred = tf.nn.xw_plus_b(fc_bn_drop, W_out, b_out, name='y_pred')  # 每个类别的分数 scores
            self._y_pred = self._y_pred+self.sim 
        

        with tf.name_scope('loss'):
            self._loss = tf.reduce_mean(
                tf.nn.sigmoid_cross_entropy_with_logits(logits=self._y_pred, labels=self._y_inputs))

    @property
    def tst(self):
        return self._tst

    @property
    def keep_prob(self):
        return self._keep_prob

    @property
    def batch_size(self):
        return self._batch_size

    @property
    def global_step(self):
        return self._global_step

    @property
    def X1_inputs(self):
        return self._X1_inputs

    @property
    def X2_inputs(self):
        return self._X2_inputs

    @property
    def y_inputs(self):
        return self._y_inputs

    @property
    def y_pred(self):
        return self._y_pred

    @property
    def loss(self):
        return self._loss

    def weight_variable(self, shape, name):
        """Create a weight variable with appropriate initialization."""
        initial = tf.truncated_normal(shape, stddev=0.1)
        return tf.Variable(initial, name=name)

    def bias_variable(self, shape, name):
        """Create a bias variable with appropriate initialization."""
        initial = tf.constant(0.1, shape=shape)
        return tf.Variable(initial, name=name)


    def lcnn_foward(self):

        question_inputs = tf.transpose(self.embedded_x_query, perm=[0, 2, 1])
        answer_inputs = tf.transpose(self.embedded_x_title, perm=[0, 2, 1])

        question_expanded = tf.expand_dims(question_inputs, -1)
        answer_expanded = tf.expand_dims(answer_inputs, -1)

        with tf.name_scope('all_pooling'):
            question_ap_0 = self.all_pool(variable_scope='input-question', x=question_expanded)
            answer_ap_0 = self.all_pool(variable_scope='input-answer', x=answer_expanded)

        question_wp_1, question_ap_1, answer_wp_1, answer_ap_1 = self.CNN_layer(variable_scope='CNN-1', x1=question_expanded, x2=answer_expanded, d=self.embedding_size)
        sims = [self.cos_sim(question_ap_0, answer_ap_0), self.cos_sim(question_ap_1, answer_ap_1)]
        # sims.append(self.get_sim(question_ap_0, answer_ap_0, self.embedding_size, 'ap_0'))
        # sims.append(self.get_sim(question_ap_1, answer_ap_1, self.di, 'ap_1'))


        if self.num_layers > 1:
            _, question_ap_2, _, answer_ap_2 = self.CNN_layer(variable_scope="CNN-2", x1=question_wp_1, x2=answer_wp_1, d=self.di)
            self.question_test = question_ap_2
            self.answer_test = answer_ap_2
            sims.append(self.cos_sim(question_ap_2, answer_ap_2))
            # sims.append(self.get_sim(question_ap_2, answer_ap_2, self.di, 'ap_2'))


        with tf.variable_scope('output_layer'):
            self.output_features = tf.stack(sims, axis=1, name='output_features')

            self.output_features = tf.layers.batch_normalization(self.output_features,training=self.training,name='lc_cnn_out')

            self.output_features = tf.nn.dropout(self.output_features, self.keep_prob, name='hidden_output_drop')

            self.fc = tf.contrib.layers.fully_connected(
                inputs = self.output_features,
                num_outputs= 256,
                activation_fn = tf.nn.tanh,
                weights_initializer=tf.contrib.layers.xavier_initializer(),
                biases_initializer=tf.constant_initializer(1e-02),
                scope="FC"
            )
            self.fc_bn = tf.layers.batch_normalization(self.fc,training=self.training,name='lc_fc_bn_')
            self.hidden_drop = tf.nn.dropout(self.fc, self.keep_prob, name='fc_drop')

            sim = tf.contrib.layers.fully_connected(
                inputs = self.hidden_drop,
                num_outputs= self.n_class,
                activation_fn = None,
                weights_initializer=tf.contrib.layers.xavier_initializer(),
                biases_initializer=tf.constant_initializer(1e-04),
                scope="FC_2"
            )


        return sim



    def CNN_layer(self, variable_scope, x1, x2, d):
        with tf.variable_scope(variable_scope):
            # 在输入层加入注意力
            if self.model_type == 'ABCNN1' or self.model_type == 'ABCNN3':
                with tf.name_scope('att_mat'):
                    # [query_len, d]
                    # question和answer共享同一个矩阵aW
                    aW = tf.get_variable(name = 'aW',
                                         shape = (self.query_len, d),
                                         initializer = tf.contrib.layers.xavier_initializer())
                    # [batch_size, query_len, query_len]
                    att_mat_A = self.make_attention_mat(x1, x2)

                    # tf.einsum("ijk,kl->ijl", att_mat_A, aW) [batch_size, query_len, d]
                    # tf.matrix_transpose(_____)  [batch_size, d, query_len]
                    # tf.expand_dims(_____)  [batch_size, d, query_len, 1]
                    x1_a = tf.expand_dims(tf.matrix_transpose(tf.einsum("ijk,kl->ijl", att_mat_A, aW)), -1)
                    x2_a = tf.expand_dims(tf.matrix_transpose(tf.einsum("ijk,kl->ijl", tf.matrix_transpose(att_mat_A), aW)), -1)
                    # [batch_size, d, query_len, 2]
                    x1 = tf.concat([x1, x1_a], axis=-1)
                    x2 = tf.concat([x2, x2_a], axis=-1)

            # 这个reuse很迷
            question_conv = self.convolution(x=self.pad_for_wide_conv(x1), d=d, reuse=False, name_scope='question')
            answer_conv = self.convolution(x=self.pad_for_wide_conv(x2), d=d, reuse=True, name_scope='answer')

            question_attention, answer_attention = None, None

            if self.model_type == 'ABCNN2' or self.model_type == 'ABCNN3':
                # matrix A [batch_size, query_len + w - 1, query_len + w - 1]
                att_mat_A = self.make_attention_mat(question_conv, answer_conv)
                # [batch_size, query_len + w - 1]
                question_attention, answer_attention = tf.reduce_sum(att_mat_A, axis=2), tf.reduce_sum(att_mat_A, axis=1)

            question_wp = self.w_pool(variable_scope='question', x=question_conv, attention=question_attention)
            question_ap = self.all_pool(variable_scope='question', x=question_conv)
            answer_wp = self.w_pool(variable_scope='answer', x=answer_conv, attention=answer_attention)
            answer_ap = self.all_pool(variable_scope='answer', x=answer_conv)

            return question_wp,question_ap,answer_wp,answer_ap

    def get_sim(self, x1, x2, d, name_scope):
        with tf.variable_scope("similarity_" + name_scope):
            M = tf.get_variable(
                name = 'M',
                shape = [d, d],
                initializer = tf.contrib.layers.xavier_initializer()
            )

            x1_trans = tf.matmul(x1, M)
            sims = tf.reduce_sum(tf.multiply(x1_trans, x2), axis=1)
            return sims


    def w_pool(self, variable_scope, x, attention):
        '''
        :param viriable_scope:
        :param x: [batch_size, di, query_len + w - 1, 1 or 2]
        :param attention: [batch_size, query_len + w -1]
        :return:
        '''
        with tf.variable_scope(variable_scope + '-w_pool'):
            if self.model_type == 'ABCNN2' or self.model_type == 'ABCNN3':
                pools = []
                # [batch, s+w-1] => [batch, s+w-1, 1, 1] => [batch, 1, s+w-1, 1]
                attention = tf.transpose(tf.expand_dims(tf.expand_dims(attention, -1), -1), [0, 2, 1, 3])
                for i in range(self.query_len):
                    pools.append(tf.reduce_sum(
                        x[:, :, i: i+self.w, :] * attention[:, :, i: i+self.w, :],
                        axis=2,
                        keepdims=True
                    ))
                w_ap = tf.concat(pools, axis=2, name='w_ap')
            else:
                w_ap = tf.layers.average_pooling2d(
                    inputs=x,
                    pool_size=(1, self.w),   #用w作为卷积窗口可以还原句子长度
                    strides=1,
                    padding='VALID',
                    name='w_ap'
                )
            # w_ap [batch_size, di, query_len, 1 or 2]
            return w_ap


    def all_pool(self, variable_scope, x):
        # al-op
        with tf.variable_scope(variable_scope + '-all_pool'):
            if variable_scope.startswith('input'):
                # 对输入层做all-pooling
                pool_width = self.query_len
                d = self.embedding_size
            else:
                # 对最后的巻积层做all-pooling
                pool_width = self.query_len + self.w - 1
                d = self.di

            all_ap = tf.layers.average_pooling2d(
                inputs = x,
                pool_size = (1, pool_width),
                strides = 1,
                padding= 'VALID',
                name = 'all_ap'
            )
            # [batch_size, di, 1, 1]

            # [batch_size, di]
            all_ap_reshaped = tf.reshape(all_ap, [-1, d])
            return all_ap_reshaped


    def make_attention_mat(self, x1, x2):
        # x1  [batch_size, embedding_size, query_len, 1]
        # tf.matrix_transpose(x2) [batch_size, embedding_size, 1, query_len]

        # 广播产生一个 [query_len_0, query_len_1]的矩阵
        # x1 - tf.matrix_transpose(x2)  [batch_size, embedding_size, query_len, query_len]
        # euclidean [bath_size, query_len, query_len]
        euclidean = tf.sqrt(tf.reduce_sum(tf.square(x1 - tf.matrix_transpose(x2)), axis=1) + self.eclipse)
        return 1 / (1 + euclidean)

    def pad_for_wide_conv(self, x):
        # 左右各填充self.w - 1个0

        # 填充前 [batch_size, d, query_len, 1] or [batch_size, d, query_len, 2]
        # 填充后 [batch_size, d, query_len - 2*(w-1), 1] or [batch_size, d, query_len - 2*(w-1), 2]
        return tf.pad(x, np.array([[0, 0], [0, 0], [self.w - 1, self.w - 1], [0, 0]]), "CONSTANT", name="pad_wide_conv")


    def convolution(self, x, d, reuse, name_scope):
        # 滑窗卷积
        with tf.name_scope(name_scope + '-conv'):
            with tf.variable_scope("conv") as scope:
                conv = tf.contrib.layers.conv2d(
                    inputs = x,
                    num_outputs = self.di,
                    kernel_size = (d, self.w),
                    stride = 1,
                    padding = 'VALID',
                    activation_fn = tf.nn.tanh,
                    weights_initializer = tf.contrib.layers.xavier_initializer_conv2d(),

                    biases_initializer = tf.constant_initializer(1e-4),
                    reuse = reuse,
                    trainable = True,
                    scope = scope,
                )
            # output [batch_size, 1, query_len + w - 1, di]

            # conv_trans: [batch_size, di, query_len + w - 1, 1]
            conv_trans = tf.transpose(conv, [0, 3, 2, 1], name = 'conv_trans')
            return conv_trans

    def cos_sim(self, v1, v2):
        norm1 = tf.sqrt(tf.reduce_sum(tf.square(v1), axis=1))
        norm2 = tf.sqrt(tf.reduce_sum(tf.square(v2), axis=1))
        dot_products = tf.reduce_sum(v1*v2, axis=1, name='cos_sim')

        return dot_products / (norm1 * norm2)
    


    def cnn_inference(self, X_inputs, n_step,name):
        """TextCNN 模型。
        Args:
            X_inputs: tensor.shape=(batch_size, n_step)
        Returns:
            title_outputs: tensor.shape=(batch_size, self.n_filter_total)
        """
        inputs = X_inputs
        inputs = tf.expand_dims(inputs, -1)
        pooled_outputs = list()
        for i, filter_size in enumerate(self.filter_sizes):
            with tf.variable_scope(("conv-maxpool-%s" % filter_size)+name):
                # Convolution Layer
                filter_shape = [filter_size, self.embedding_size, 1, self.n_filter]
                W_filter = self.weight_variable(shape=filter_shape, name='W_filter')
                beta = self.bias_variable(shape=[self.n_filter], name='beta_filter')
                conv = tf.nn.conv2d(inputs, W_filter, strides=[1, 1, 1, 1], padding="VALID", name="conv")
                conv_bn=tf.layers.batch_normalization(conv,training=self.training,name=name)
                # Apply nonlinearity, batch norm scaling is not useful with relus
                # batch norm offsets are used instead of biases,使用 BN 层的 offset，不要 biases
                h = tf.nn.relu(conv_bn, name="relu")
                # Maxpooling over the outputs
                pooled = tf.nn.max_pool(h, ksize=[1, n_step - filter_size + 1, 1, 1],
                                        strides=[1, 1, 1, 1], padding='VALID', name="pool")
                pooled_outputs.append(pooled)
        h_pool = tf.concat(pooled_outputs, 3)
        h_pool_flat = tf.reshape(h_pool, [-1, self.n_filter_total])
        return h_pool_flat  # shape = [batch_size, self.n_filter_total]



def loader(data_store_path=''):############
    # all_data=pd.read_csv(data_store_path+"data_sets/train_feature_second_no_query_and_title_data.csv")
    # query_title=pd.read_csv(data_store_path+"data_sets/train_data.csv",usecols=['query','title'])
    print("开始加载数据")
    all_data=pd.read_pickle(data_store_path+"/home/kesci/train_data0-5000w.pickle")
    data_split=StratifiedShuffleSplit(n_splits=2,test_size=0.001,random_state=1)
    train_index,vaild_index=data_split.split(all_data['label'],all_data['label']).__next__()
    train_data=all_data.iloc[train_index]
    vaild_data=all_data.iloc[vaild_index]
    print("加载数据完成")
    return train_data,vaild_data

def batch_iter(data_x,data_y, batch_size, num_epochs, shuffle=True):#这个就是产生batch的，可以直接用
    """
    Generates a batch iterator for a dataset.
    """
    # data_x=data_x.tolist()
    data=list(zip(data_x,data_y))
    data=np.array(data)
    data_size = len(data)
    num_batches_per_epoch = int((len(data)-1)/batch_size) + 1
    for epoch in range(num_epochs):
        # Shuffle the data at each epoch
        if shuffle:#如果为true 则代表允许随机取数据
            shuffle_indices = np.random.permutation(np.arange(data_size))
            shuffled_data = data[shuffle_indices]#随机取数据
        else:
            shuffled_data = data
        for batch_num in range(num_batches_per_epoch):
            start_index = batch_num * batch_size
            end_index = min((batch_num + 1) * batch_size, data_size)
            yield shuffled_data[start_index:end_index]#而不用一个个加到列表里了。这是一个batch，嗯哼
            #取到的维度为（batchsize,2） y为(batchsize,1) 里面存的是序号

from gensim import models
import pickle
fw=open('dictionary/dictionary.pickle','rb')
dictionary=pickle.load(fw)
import pickle
fw=open('tfidf_model/tfidf_model.pickle','rb')
tfidf_model=pickle.load(fw)

def get_w2v_array(word_list,max_len):
    single_sentence_tfidf=tfidf_model[dictionary.doc2bow(word_list)]
    # single_sentence_tfidf={dictionary[word_id_tfidf[0]]:word_id_tfidf[1] for word_id_tfidf in single_sentence_tfidf}
    array = np.zeros((max_len, 100))
    if len(word_list)<=max_len:
        for i in range(len(word_list)):
            if str(word_list[i]) in word_wv.vocab.keys():
                array[i][:] = word_wv[str(word_list[i])]
                # try:
                #     array[i][:]*=single_sentence_tfidf[str(word_list[i])]

                # except:
                #     pass
                
    else:
        for i in range(max_len):
            if str(word_list[i]) in word_wv.vocab.keys():
                array[i][:] = word_wv[str(word_list[i])]
                # try:
                #     array[i][:]*=single_sentence_tfidf[str(word_list[i])]
                # except:
                #     pass
    return array

settings = Settings()

train_batches,vaild_data=loader()

train_batches=np.array(train_batches)
train_batches_y=train_batches[:,2]
train_batches_x=np.delete(train_batches, 2, axis=1)######这里是除了label以外的维数


batches=batch_iter(train_batches_x,train_batches_y,settings.train_batch_size,4)

vaild_batches=np.array(vaild_data)
vaild_batches_y=vaild_batches[:,2]
vaild_batches_x=np.delete(vaild_batches, 2, axis=1)
vaild_data=batch_iter(vaild_batches_x,vaild_batches_y,settings.vaild_batch_size,1000)

print("train starting")
w2v_model = Word2Vec.load('w2v_model3/w2v_all_data_model.txt')
word_wv= w2v_model.wv


settings = Settings()

settings.all_features_num=train_batches_x.shape[1]-2###########自动设置数值型特征数量
model = TEXTCNN( settings)

learning_rate = tf.train.exponential_decay(settings.lr, model.global_step, settings.decay_step,
                                       settings.decay_rate, staircase=True)
optimizer = tf.train.AdamOptimizer(learning_rate)

update_ops=tf.get_collection(tf.GraphKeys.UPDATE_OPS)
with tf.control_dependencies(update_ops):
    grads_and_vars=optimizer.compute_gradients(model.loss)
    train_op=optimizer.apply_gradients(grads_and_vars,global_step=model.global_step)

config = tf.ConfigProto() 
config.gpu_options.allow_growth = True 
sess = tf.Session(config=config) #启动创建的模型
# sess.run(tf.initialize_all_variables())

# saver = tf.train.Saver(tf.global_variables())
saver = tf.train.Saver()
sess.run(tf.global_variables_initializer())
# saver.restore(sess, tf.train.latest_checkpoint('/home/kesci/work/tf_model_textcnn/'))
per_train_loss_avg_list=[]
per_vaild_loss_avg_list=[]
print("start")
parts_time=[]
for batch in batches:
    from time import time
    time1=time()
    batch_x,batch_y=zip(*batch)
    batch_y=np.array(batch_y)[:,None]
    batch_x=[i.tolist() for i in batch_x]
    batch_x=pd.DataFrame(batch_x)

    batch_x[[0,1]]=batch_x[[0,1]].applymap(lambda x:x.split(' '))
    batch_x=np.array(batch_x)
    #####[None,title_len,embedding_size]
    #np.apply_along_axis(lambda x:[5,5,5,5,5] if x[0]==2 else x,axis=2,arr=a)
    #query   title   title_count         title_ctr      label
    batch_x_query=np.apply_along_axis(lambda x:get_w2v_array(x[0],settings.query_len),axis=1,arr=batch_x)
    batch_x_title=np.apply_along_axis(lambda x:get_w2v_array(x[1],settings.title_len),axis=1,arr=batch_x)

        
    #     train_op=optimizer.apply_gradients(grads_and_vars,global_step=global_step)                              
    feed_dict = {model.embedded_x_query: batch_x_query,
                 model.embedded_x_title: batch_x_title,
                 model.y_inputs: batch_y,
                 model.batch_size: batch_x.shape[0], 
                 model.training: True, 
                 model.keep_prob: 0.8}
    
    _,step,loss_out=sess.run([train_op,model.global_step,model.loss],feed_dict)
    time2=time()
    parts_time.append(time2-time1)
    if step%10==0:
        print("step {},loss:{:g}".format(step,loss_out))              
    per_train_loss_avg_list.append(loss_out)
    
    if step%(int(50000000/1004))==0:
        print("batch ended")
    if step%500==0:
        time_single_part=sum(parts_time)
        ed_time=(50000000/settings.train_batch_size-step)/500*time_single_part
        print("距离训练完一整轮剩余时间：",ed_time/60," 分钟")
        parts_time=[]
        vaild_x,vaild_y=zip(*(vaild_data.__next__()))
        vaild_y=np.array(vaild_y)[:,None]
        vaild_x=[i.tolist() for i in vaild_x]
        vaild_x=pd.DataFrame(vaild_x)
        vaild_x[[0,1]]=vaild_x[[0,1]].applymap(lambda x:x.split(' '))
        vaild_x=np.array(vaild_x)
    
        #####[None,title_len,embedding_size]
        #np.apply_along_axis(lambda x:[5,5,5,5,5] if x[0]==2 else x,axis=2,arr=a)
        #query   title   title_count         title_ctr      label
        vaild_x_query=np.apply_along_axis(lambda x:get_w2v_array(x[0],settings.query_len),axis=1,arr=vaild_x)
        vaild_x_title=np.apply_along_axis(lambda x:get_w2v_array(x[1],settings.title_len),axis=1,arr=vaild_x)


        # #########################这是连续型特征
        # vaild_x_ctr_fea=vaild_x[:,2:]######这里需要到时候调整，按label的位置和特征位置
        # vaild_x_ctr_fea=np.delete(vaild_x_ctr_fea, -1, axis=1)######需要把类别型特征去掉
        # ##########################
        # ###########################这是类别型特征
        # vaild_x_tfidf=vaild_x[:,-1][:,None]
        # #########################################


        feed_dict = {model.embedded_x_query: vaild_x_query,
                     model.embedded_x_title: vaild_x_title,
                     model.y_inputs: vaild_y,
                     model.batch_size: vaild_x.shape[0], 
                     model.training: False, 
                     model.keep_prob: 1}
        
        
        step,loss_out=sess.run([model.global_step,model.loss],feed_dict)
        per_vaild_loss_avg_list.append(loss_out)
        per_train_loss_avg=sum(per_train_loss_avg_list)/len(per_train_loss_avg_list)
        print("per_train_loss_avg:",per_train_loss_avg,"**********")
        per_train_loss_avg_list=[]
        print("vaild_loss:",loss_out)
    if step%5000==0:
        per_vaild_loss_avg=sum(per_vaild_loss_avg_list)/len(per_vaild_loss_avg_list)
        per_vaild_loss_avg_list=[]
        print("per_vaild_loss_avg:",per_vaild_loss_avg,"************")
        saver = tf.train.Saver(max_to_keep = 3,var_list=tf.global_variables())
        saver = saver.save(sess, "/home/kesci/work/tf_model_textcnn_rcnn/cnn", global_step=step)

开始加载数据
加载数据完成
train starting


/opt/conda/lib/python3.6/site-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


start
step 10,loss:1.18032
step 20,loss:0.971862
step 30,loss:0.537315
step 40,loss:0.425747
step 50,loss:0.302233
step 60,loss:0.351071
step 70,loss:0.21517
step 80,loss:0.234336
